# Figure 2 in Hjorth et al, 2021 Neuroinformatics paper

This code can be run in parallel. To do that, please first start ```ipcluster```, on linux you write the following into the terminal, note that you should be in the same directory as the notebook is in (```examples/Neurinformatics2021/Figure2```):

```
export IPYTHONDIR="`pwd`/.ipython"
export IPYTHON_PROFILE=default
ipcluster start 
```

These commands are in ```start_ipcluster.sh``` in the ```examples/notebook``` directory. After starting ipcluster give it a few seconds to get fully up and running before proceeding with the rest of the commands in this notebook. 

If you restart the run you might need to do:

```
ipcluster stop
ipcluster start
```

For instructions how to install and activate ipyparallel for your notebook see 
https://github.com/ipython/ipyparallel

In [ ]:
import os
from ipyparallel import Client

os.environ["IPYTHONDIR"] = ".ipython"
os.environ["IPYTHON_PROFILE"] = "default"

u_file = os.path.join(".ipython", "profile_default", "security", "ipcontroller-client.json")
if os.path.exists(u_file):
    rc = Client(url_file=u_file, timeout=120, debug=False)
else:
    print("Warning, running in serial.")
    rc = None

In [ ]:
from methods_paper_figure2 import MethodsPaperFigure2
from snudda import SnuddaPlace
from snudda import SnuddaDetect
from snudda import SnuddaPrune

network_type = ['No pruning',
                'DP',
                'DP, f1',
                'DP, f1, SM',
                'DP, f1, SM, mu2']

network_size = 10000
network_path = "Fig2_network"
config_list = []
network_file_list = []

for idx, nt in enumerate(network_type):
    config_name = os.path.join(f"network-config-{idx}.json")
    config_list.append(config_name)
    network_file = os.path.join(network_path, f"network-synapses-{idx}.hdf5")
    network_file_list.append(network_file)

In [ ]:
for config_name, nt in zip(config_list, network_type):
    MethodsPaperFigure2.setup_network(network_path=network_path, config_name=config_name,
                                      network_type=nt, n_neurons=network_size, random_seed=1234)

In [ ]:
conf_file=os.path.join(network_path, config_list[-1])

sp = SnuddaPlace(config_file=conf_file, rc=rc)
sp.place()
del sp
    
sd = SnuddaDetect(config_file=conf_file, rc=rc)
sd.detect()
del sd  # Clean up memory

In [ ]:
network_file = os.path.join(network_path, "network-synapses.hdf5")
    
for config_name, new_network_file in zip(config_list, network_file_list):
    config_file = os.path.join(network_path, config_name)
    print(f"Using config_file={config_file}")
    sp2 = SnuddaPrune(network_path=network_path, config_file=config_file, rc=rc, keep_files=True)
    sp2.prune()
    del sp2
    
    # We need to save the network files with new names for each pruning
    os.rename(network_file, new_network_file)

In [ ]:
mpf = MethodsPaperFigure2(network_file_list=network_file_list)
mpf.make_connection_probability_summary('iSPN', 'dSPN')
mpf.make_num_synapses_summary_figure('iSPN', 'dSPN')
mpf.summary_plot_cum_dist('iSPN', 'dSPN')